# Accompanying code to replication and extension of *Progress Measures for Grokking via Mechanistic Interpretability* paper

This notebook represents my personal project of replicating the *Progress Measures for Grokking via Mechanistic Interpretability* research paper by Neel Nanda et al. published in Oct 2023.
<br>
The initial focus will be on training the mainline model from the paper and reproducing the Grokking behavior.
<br>
The next step will be changing the dataset and repeating the process.
<br>
Finally, I will explore the mechanistic interpretability part of the work and dig into the model modifications to see if I'm able to learn a few things about this process and maybe discover some new interesting things along the way.

To use this notebook, T4 GPU compute resources have to be selected from the Runtime -> Change runtime type -> T4 GPU menu when using this notebook in Google Colab.

The plotly and kaleido libraries used in this notebook for plotting and downloading graphs are not version controlled necessarily. When you encounter issues specifically relating to these libraries, you may need to change the versions.

## Setup Code

We are first going to download packages. We will then import the necessary libraries.

In [ ]:
# These libraries require to restart session: Run this cell -> restart session -> run cell again
# Installing specific kaleido package for plotly image export
!pip install --upgrade "kaleido==0.1.*" # May need to change version if dependencies change

# Transformer_lens needed to gain access to simple transformer architecture (allows disabling LayerNorm)
!pip install transformer_lens

In [ ]:
# Use this library to load files into google colab from google drive
from google.colab import drive

# Standard libaries for ML
import einops
import torch as t
import numpy as np
from copy import deepcopy
from tqdm.auto import tqdm
import torch.optim as optim
from dataclasses import dataclass

# Custom libaries for Transformer architecture
import transformer_lens.utils as utils
from transformer_lens import HookedTransformerConfig, HookedTransformer

# Plotting library
import plotly.express as px

### Configuration

Next we are going to build a configuration dataclass that stores all the hyperparameters. We are using a frozen=True keyword to lock all parameters in place and make the config file immutable.

In [ ]:
# Configuration dataclass that stores all the important experimental hyperparameters
@dataclass(frozen=True)
class Config():
    p: int = 113 # prime modulus
    training_frac: float = 0.3 # ratio of train / test split
    training_index: int = round(p*p*training_frac) # index of dataset to split train / test
    lr: float = 1e-3 # learning rate of AdamW optimizer
    wd: float = 1.0 # weight decay of AdamW optimizer
    betas: tuple[float] = (0.9, 0.98) # Momentum and adaptive gradient beta-values for AdamW optimizer
    epochs: int = 20_000 # number of epochs
    checkpoint_every: int = 100 # how often to record model checkpoints
    device: str = "cuda" if t.cuda.is_available() else "cpu" # select GPU to store tensors, or CPU when GPU is not available
    seed: int = 1234 # seed for experiment to generate pseudo-random numbers for training and testing data
    model_seed: int = 999 # seed for model, change this value to run multiple times

In [ ]:
# Instantiating configuration
config = Config()

The following configuration file is adapted specifically for the dual task experiment (training a single model on two different algorithmic tasks at once)
training_index = round(2 * p * p * training_frac) instead of round(p * p * training_frac)

In [ ]:
# Configuration dataclass that stores all the important experimental hyperparameters
@dataclass(frozen=True)
class Config2():
    p: int = 79 # prime modulus
    training_frac: float = 0.3 # ratio of train / test split
    training_index: int = round(2*p*p*training_frac) # index of dataset to split train / test
    lr: float = 1e-3 # learning rate of AdamW optimizer
    wd: float = 1.0 # weight decay of AdamW optimizer
    betas: tuple[float] = (0.9, 0.98) # Momentum and adaptive gradient beta-values for AdamW optimizer
    epochs: int = 20_000 # number of epochs
    checkpoint_every: int = 100 # how often to record model checkpoints
    device: str = "cuda" if t.cuda.is_available() else "cpu" # select GPU to store tensors, or CPU when GPU is not available
    seed: int = 1234 # seed for experiment to generate pseudo-random numbers for training and testing data
    model_seed: int = 999 # seed for model, change this value to run multiple times

In [ ]:
# Instantiating configuration for dual task problem
config2 = Config2()

The following configuration is for the transformer model. We will use the config constructor class from transformer_lens and pass in all the model architecture hyperparameters. The following hyperparameters are from the mainline experimental model.

In [ ]:
# Creating model configuration using data from mainline experiment
model_cfg = HookedTransformerConfig(
    n_layers=1, # Number of layers
    d_model=128, # Model dimension
    n_ctx=3, # Three input-tokens (a, b and equals-sign)
    d_head=32, # Attention head dimension (d_model // n_heads)
    n_heads=4, # Number of attention heads
    d_mlp=512, # MLP dimension
    act_fn="relu", # MLP activation function
    d_vocab=config.p+1, # Vocabulary size (p characters + equals-sign)
    d_vocab_out=config.p, # Vocabulary size out (p characters, number of possible answers for equation)
    normalization_type=None, # No LayerNorm
    init_weights=True, # Weight initialization
    device="cuda" if t.cuda.is_available() else "cpu", # Model on GPU
    seed=config.model_seed # Change this particular value when re-running experiment for different seeds
)

The below model configuration should specifically be used for the dual task problem. The two parameters that are changed are the context tokens (3 to 4) and the input vocabulary (P+1 to P+3)

In [ ]:
# Creating model configuration for dual task problem
model_cfg_2eq = HookedTransformerConfig(
    n_layers=1, # Number of layers
    d_model=128, # Model dimension
    n_ctx=4, # Four input-tokens (a, b, operator and equals-sign)
    d_head=32, # Attention head dimension (d_model // n_heads)
    n_heads=4, # Number of attention heads
    d_mlp=512, # MLP dimension
    act_fn="relu", # MLP activation function
    d_vocab=config2.p+3, # Vocabulary size (p characters + equals-sign + two operators)
    d_vocab_out=config2.p, # Vocabulary size out (p characters, number of possible answers for equation)
    normalization_type=None, # No LayerNorm
    init_weights=True, # Weight initialization
    device="cuda" if t.cuda.is_available() else "cpu", # Model on GPU
    seed=config2.model_seed # Change this particular value when re-running experiment for different seeds
)

### Custom functions
Including: custom loss, training function, plotting functions, and other helper functions.

This next section displays all the custom functions used throughout this notebook. These functions can be altered to change implementation and workflow.

Cross-entropy loss function from PyTorch is not used, because this function is optimized for CNNs. I am using Neel's implementation here.

In [ ]:
# Custom loss function
def loss_fn(logits, labels):
    if len(logits.shape) == 3:
        logits = logits[:, -1] # Return only last token of three token index ("=" sign), automatically convert to 2 dimensional tensor
    logits = logits.to(t.float64) # Numerical stability to prevent spikes in loss
    logprobs = logits.log_softmax(dim=-1)
    correct_logprobs = logprobs.gather(dim=-1, index=labels[:, None])[:, 0]
    return -correct_logprobs.mean()

In [ ]:
# Custom training function to train transformer (includes checkpoint generation)
def train(model, train_data, train_labels, test_data, test_labels, loss_fn, config):
    train_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []
    model_checkpoints = []
    checkpoint_epochs = []

    for epoch in tqdm(range(config.epochs)):
        # Training
        train_logits = model(train_data)
        loss = loss_fn(train_logits, train_labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Record train loss
        train_losses.append(loss.item())

        # Train accuracy
        train_logits_mod = train_logits[:, -1]
        train_probs = train_logits_mod.softmax(dim=-1)
        accuracy = ((train_probs.argmax(dim=-1) == train_labels).sum() / train_data.shape[0]).item() # Argmax accuracy
        train_accuracies.append(accuracy)

        # Testing / validation
        with t.inference_mode():
            test_logits = model(test_data)
            test_loss = loss_fn(test_logits, test_labels)

            # Record test loss
            test_losses.append(test_loss.item())

            # Test accuracy
            test_logits_mod = test_logits[:, -1]
            test_probs = test_logits_mod.softmax(dim=-1)
            test_accuracy = ((test_probs.argmax(dim=-1) == test_labels).sum() / test_data.shape[0]).item()
            test_accuracies.append(test_accuracy)

        # Record model checkpoints and state_dict and output data in terminal
        if ((epoch+1)%config.checkpoint_every) == 0:
            checkpoint_epochs.append(epoch)
            model_checkpoints.append(deepcopy(model.state_dict()))
            print(f"Epoch {epoch} Train loss {loss.item()} Test loss {test_loss.item()}")

    return train_losses, test_losses, train_accuracies, test_accuracies, model_checkpoints, checkpoint_epochs

Plotting functions

In [ ]:
# Custom plotting helper function using transformer_lens library extension -> plotly express line function
# Do not use the x-parameter in this function as that will break the plotting!!!
def line(
        tensor,
        xaxis="x",
        yaxis="y",
        line_labels=None,
        x_labels=None,
        x_range=None,
        plot_average=False,
        show_seeds=True,
        legend_dict=None,
        line_colors=None,
        start_epoch=0,
        **kwargs
):

    # Build multiple graphs in one plot
    if isinstance(tensor, list):
        data = np.stack([utils.to_numpy(t) for t in tensor], axis=1)
    else:
        data = utils.to_numpy(tensor)

    # Base figure with bold axis titles
    fig = px.line(
        data,
        labels={"index": f"<b>{xaxis}</b>", "value": f"<b>{yaxis}</b>"},
        **kwargs
    )

    # Remove legend title
    fig.update_layout(
        legend_title_text=""
    )

    # Add average curves using plot_average keyword
    if plot_average:

        n_points, n_series = data.shape

        # Assume first half = train, second half = test
        half = n_series // 2
        train_indices = list(range(half))
        test_indices = list(range(half, n_series))

        for i, trace in enumerate(fig.data):
            if i in train_indices:
                trace.update(line=dict(color="blue"), opacity=0.5)
            else:
                trace.update(line=dict(color="#d97706"), opacity=0.5)

            if not show_seeds:
                trace.visible = "legendonly"

        train_avg = data[:, train_indices].mean(axis=1)
        test_avg = data[:, test_indices].mean(axis=1)

        fig.add_scatter(
            x=list(range(n_points)),
            y=train_avg,
            mode="lines",
            line=dict(color="blue")
        )

        fig.add_scatter(
            x=list(range(n_points)),
            y=test_avg,
            mode="lines",
            line=dict(color="#d97706")
        )

        # Hide seed curves from legend entirely
        for trace in fig.data[:n_series]:
            trace.showlegend = False

        # Modify the legend line labels
        if line_labels is not None:
            for i, trace in enumerate(fig.data[n_series:]):
                trace.update(name=line_labels[i])

    else:
        # Modify the legend line labels
        if line_labels is not None:
            for i, trace in enumerate(fig.data):
                trace.update(name=line_labels[i])

    # Modify legend position
    if legend_dict is not None:
        fig.update_layout(legend=legend_dict)

    # Modify tick labels
    if x_labels is not None:
        fig.update_xaxes(
            tickmode="array",
            tickvals=list(range(len(x_labels))),
            ticktext=x_labels
        )

    # Modify x-axis stretching based on data upload and data display needs
    if x_range is not None:
        n_points, n_series = data.shape
        scale = x_range / (n_points - 1)
        for trace in fig.data:
            trace.x = [start_epoch + i * scale for i in range(n_points)]

    # Modify line colors
    if line_colors is not None:
        for i, trace in enumerate(fig.data):
            trace.update(line=dict(color=line_colors[i]))

    return fig

In [ ]:
# Custom plotting helper function using transformer_lens library extension -> plotly express bar function
# Do not use the x-parameter in this function as that will break the plotting!!!
def bar(
        tensor,
        xaxis="x",
        yaxis="y",
        bar_labels=None,
        legend_dict=None,
        bar_colors=None,
        **kwargs
):

    # Build multiple graphs in one plot
    if isinstance(tensor, list):
        data = np.stack([utils.to_numpy(t) for t in tensor], axis=1)
    else:
        data = utils.to_numpy(tensor)

    # Base figure with bold axis titles
    fig = px.bar(
        data,
        labels={"index": f"<b>{xaxis}</b>", "value": f"<b>{yaxis}</b>"},
        **kwargs
    )

    # Remove legend title and set barmode
    fig.update_layout(
        barmode="group",
        legend_title_text=""
    )

    # Modify the legend line labels
    if bar_labels is not None:
        for i, trace in enumerate(fig.data):
            trace.update(name=bar_labels[i])

    # Modify legend position
    if legend_dict is not None:
        fig.update_layout(legend=legend_dict)

    # Modify line colors
    if bar_colors is not None:
        for i, trace in enumerate(fig.data):
            trace.update(marker=dict(color=bar_colors[i]))

    return fig

In [ ]:
# Custom plotting helper functions using transformer_lens library extensions -> imshow heatmap function
def imshow(tensor, xaxis="x", yaxis="y", **kwargs):
    fig = px.imshow(
        utils.to_numpy(tensor),
        color_continuous_midpoint=0.0,
        color_continuous_scale="RdBu",
        labels={"x": f"<b>{xaxis}</b>", "y": f"<b>{yaxis}</b>"},
        **kwargs
    )

    # Prevent figure from forced square geometry
    fig.update_xaxes(scaleanchor=None)
    fig.update_yaxes(scaleanchor=None)

    return fig

In [ ]:
# Custom function to add vertical lines to a plot
def add_vlines(fig, xpos):

    # Add vertical lines
    for x in xpos:
        fig.add_vline(
        x=x,
        line_width=2,
        line_dash="dash",
        opacity=0.5
        )

    return fig

Saving and loading data functions

In [ ]:
# Custom function to save plots
def save_fig(fig, name, scale=3):
    fig.write_image(f"{name}.png", width=fig.layout.width, height=fig.layout.height, scale=scale)

Helper functions for progress measures

In [ ]:
# Custom function to run a metric over every checkpoint of the model
def get_metrics(model, model_checkpoints, metric_cache, metric_fn, *args, name="", reset=False):
    if reset or (name not in metric_cache) or (len(metric_cache[name]) == 0):
        metric_cache[name] = []
        for sd in tqdm((model_checkpoints)):
            model.reset_hooks()
            model.load_state_dict(sd)
            out = metric_fn(model, *args)
            if type(out) == t.Tensor:
                out = utils.to_numpy(out)
            metric_cache[name].append(out)
        model.load_state_dict(model_checkpoints[-1])
        try:
            metric_cache[name] = t.tensor(metric_cache[name])
        except:
            metric_cache[name] = t.tensor(np.array(metric_cache[name]))

In [ ]:
# Get coefficients of cosine similarity
def get_cos_coeffs(model, cos_cube, dataset, config):
    logits = model(dataset)[:, -1]
    logits = einops.rearrange(logits, "(a b) c -> a b c", a=config.p, b=config.p)
    vals = (cos_cube * logits[None, :, :, :]).sum([-3, -2, -1])
    return vals

In [ ]:
# Custom function for restricted loss progress measure
def get_restricted_loss(model, dataset, key_freqs, index, labels, config):
    logits, cache = model.run_with_cache(dataset)
    logits = logits[:, -1, :]
    neuron_acts = cache["post", 0, "mlp"][:, -1, :]
    approx_neuron_acts = t.zeros_like(neuron_acts)
    approx_neuron_acts += neuron_acts.mean(dim=0)
    a = t.arange(config.p)[:, None]
    b = t.arange(config.p)[None, :]
    for freq in key_freqs:
        cos_apb_vec = t.cos(freq * 2 * t.pi / config.p * (a + b)).to(config.device)
        cos_apb_vec /= cos_apb_vec.norm()
        cos_apb_vec = einops.rearrange(cos_apb_vec, "a b -> (a b) 1")
        approx_neuron_acts += (neuron_acts * cos_apb_vec).sum(dim=0) * cos_apb_vec
        sin_apb_vec = t.sin(freq * 2 * t.pi / config.p * (a + b)).to(config.device)
        sin_apb_vec /= sin_apb_vec.norm()
        sin_apb_vec = einops.rearrange(sin_apb_vec, "a b -> (a b) 1")
        approx_neuron_acts += (neuron_acts * sin_apb_vec).sum(dim=0) * sin_apb_vec
    restricted_logits = approx_neuron_acts @ model.blocks[0].mlp.W_out @ model.unembed.W_U
    # Add bias term
    restricted_logits += logits.mean(dim=0, keepdim=True) - restricted_logits.mean(dim=0, keepdim=True)
    return loss_fn(restricted_logits[index[config.training_index:]], labels[index[config.training_index:]])

In [ ]:
# Custom function for excluded loss progress measure
def get_excluded_loss(model, dataset, key_freqs, index, labels, config):
    logits, cache = model.run_with_cache(dataset)
    logits = logits[:, -1, :]
    neuron_acts = cache["post", 0, "mlp"][:, -1, :]
    approx_neuron_acts = t.zeros_like(neuron_acts)
    # approx_neuron_acts += neuron_acts.mean(dim=0)
    a = t.arange(config.p)[:, None]
    b = t.arange(config.p)[None, :]
    for freq in key_freqs:
        cos_apb_vec = t.cos(freq * 2 * t.pi / config.p * (a + b)).to(config.device)
        cos_apb_vec /= cos_apb_vec.norm()
        cos_apb_vec = einops.rearrange(cos_apb_vec, "a b -> (a b) 1")
        approx_neuron_acts += (neuron_acts * cos_apb_vec).sum(dim=0) * cos_apb_vec
        sin_apb_vec = t.sin(freq * 2 * t.pi / config.p * (a + b)).to(config.device)
        sin_apb_vec /= sin_apb_vec.norm()
        sin_apb_vec = einops.rearrange(sin_apb_vec, "a b -> (a b) 1")
        approx_neuron_acts += (neuron_acts * sin_apb_vec).sum(dim=0) * sin_apb_vec
    excluded_neuron_acts = neuron_acts - approx_neuron_acts
    residual_stream_final = excluded_neuron_acts @ model.blocks[0].mlp.W_out + cache["resid_mid", 0][:, -1, :]
    excluded_logits = residual_stream_final @ model.unembed.W_U
    return loss_fn(excluded_logits[index[:config.training_index]], labels[index[:config.training_index]])

This method is needed for the cosine similarity metrics (technically not a function)

In [ ]:
# Cosine similarity metric used for logit periodicity progress measure
cos_cube = []
for freq in range(1, config.p//2 + 1):
    a = t.arange(config.p)[:, None, None]
    b = t.arange(config.p)[None, :, None]
    c = t.arange(config.p)[None, None, :]
    cube_predicted_logits = t.cos(freq * 2 * t.pi / config.p * (a + b - c)).to(config.device)
    cube_predicted_logits /= cube_predicted_logits.norm()
    cos_cube.append(cube_predicted_logits)
cos_cube = t.stack(cos_cube, dim=0)

These versions are needed for the dual task problem

In [ ]:
# Cosine similarity metric used for logit periodicity progress measure
cos_cube2 = []
for freq in range(1, config2.p//2 + 1):
    a = t.arange(config2.p)[:, None, None]
    b = t.arange(config2.p)[None, :, None]
    c = t.arange(config2.p)[None, None, :]
    cube_predicted_logits = t.cos(freq * 2 * t.pi / config2.p * (a + b - c)).to(config2.device)
    cube_predicted_logits /= cube_predicted_logits.norm()
    cos_cube2.append(cube_predicted_logits)
cos_cube2 = t.stack(cos_cube2, dim=0)

In [ ]:
# Cosine similarity metric used for logit periodicity progress measure (using modular subtraction)
cos_cube3 = []
for freq in range(1, config2.p//2 + 1):
    a = t.arange(config2.p)[:, None, None]
    b = t.arange(config2.p)[None, :, None]
    c = t.arange(config2.p)[None, None, :]
    cube_predicted_logits = t.cos(freq * 2 * t.pi / config2.p * (a - b - c)).to(config2.device)
    cube_predicted_logits /= cube_predicted_logits.norm()
    cos_cube3.append(cube_predicted_logits)
cos_cube3 = t.stack(cos_cube3, dim=0)

Fourier basis for mechanistic interpretability (technically not a function, but storing here for organizational purposes)

In [ ]:
# Create fourier basis using sine and cosine functions (used the implementation from the GitHub repo:
# https://github.com/mechanistic-interpretability-grokking/progress-measures-paper/blob/main/Grokking_Analysis.ipynb)
fourier_basis = []
fourier_basis_names = ["Constant"]
fourier_basis.append(t.ones(config.p) / np.sqrt(config.p))
for freq in range(1, config.p//2+1): # Beyond this frequency, things are symmetric
    fourier_basis.append(t.cos((t.arange(config.p) * 2 * t.pi * freq) / config.p))
    fourier_basis.append(t.sin((t.arange(config.p) * 2 * t.pi * freq) / config.p))
    fourier_basis[-2]/=fourier_basis[-2].norm()
    fourier_basis[-1]/=fourier_basis[-1].norm()
    fourier_basis_names.append(f"cos {freq}")
    fourier_basis_names.append(f"sin {freq}")
fourier_basis = t.stack(fourier_basis, dim=0).to(config.device) # Stack cosine and sine waves together

This version is needed for the dual task problem

In [ ]:
# Create fourier basis using sine and cosine functions (used the implementation from the GitHub repo:
# https://github.com/mechanistic-interpretability-grokking/progress-measures-paper/blob/main/Grokking_Analysis.ipynb)
fourier_basis2 = []
fourier_basis_names2 = ["Constant"]
fourier_basis2.append(t.ones(config2.p) / np.sqrt(config2.p))
for freq in range(1, config2.p//2+1): # Beyond this frequency, things are symmetric
    fourier_basis2.append(t.cos((t.arange(config2.p) * 2 * t.pi * freq) / config2.p))
    fourier_basis2.append(t.sin((t.arange(config2.p) * 2 * t.pi * freq) / config2.p))
    fourier_basis2[-2]/=fourier_basis2[-2].norm()
    fourier_basis2[-1]/=fourier_basis2[-1].norm()
    fourier_basis_names2.append(f"cos {freq}")
    fourier_basis_names2.append(f"sin {freq}")
fourier_basis2 = t.stack(fourier_basis2, dim=0).to(config2.device) # Stack cosine and sine waves together

### Creating filepath to save data

In [ ]:
# Create file path
BASE = "/content/"
RUN = "filename.pth" # Modify this parameter after every new run, choose your own format
PTH_LOCATION = f"{BASE}{RUN}"

PTH_LOCATION

## Creating Dataset

Modular addition:
<br>
Model takes input $ a,b \in \{0, ..., P - 1\} $ for some prime $ P $ and predicts the sum $ c \mod P $.
<br>
<br>
Datasets & labels:
<br>
Input to the model should be of the form $ "a \text{ } b =" $, which means the model will receive three input tokens (one for a, one for b, and one for the "=" sign). For the mainline experiment, $ P = 113 $.

The following code was copied from Neel Nanda's instruction video (https://www.youtube.com/watch?v=ob4vuiqG2Go&list=PL7m7hLIqA0hqsReJ0NYhyN3xiFQW9ko1h&index=2)

In [ ]:
a_vector = einops.repeat(t.arange(config.p), "i -> (i j)", j=config.p) # shape (P*P,)
b_vector = einops.repeat(t.arange(config.p), "j -> (i j)", i=config.p) # shape (P*P,)
equals_vector = einops.repeat(t.tensor(config.p), " -> (i j)", i=config.p, j=config.p) # shape (P*P,)

In [ ]:
dataset = t.stack((a_vector, b_vector, equals_vector), dim=1).to(config.device) # shape(P*P, 3)

dataset[:5], dataset.shape

In [ ]:
labels = (dataset[:, 0] + dataset[:, 1]) % config.p # Modular addition formula "a + b mod P"

labels[:5], labels.shape

Train / test split

In [ ]:
# Use training_frac split criteria and randomization of the dataset and labels (avoid particular pattern in training or testing)
t.manual_seed(config.seed) # Use pseudorandomness
index = t.randperm(config.p*config.p) # shape (P*P,)
random_dataset = dataset[index] # randomized dataset
random_labels = labels[index]
train_data, train_labels = random_dataset[:config.training_index], random_labels[:config.training_index]
test_data, test_labels = random_dataset[config.training_index:], random_labels[config.training_index:]

train_data.shape, train_labels.shape, test_data.shape, test_labels.shape, \
train_data[:5], train_labels[:5], test_data[:5], test_labels[:5]

## Creating Two Equation Dataset
Modular addition and modular subtraction:
<br>
Model takes input $ a,b \in \{0, ..., P - 1\} $ for some prime $ P $ and predicts both the sum $ c \mod P $ and the difference $ d \mod P $.
<br>
<br>
Datasets & labels:
<br>
Input to the model should be of the form $ "a + b =" or "a - b =" $, which means the model will receive four input tokens (one for a, one for b, one for the operator, and one for the "=" sign). For the main experiment, $ P = 79 $.

In [ ]:
a_vector2 = einops.repeat(t.arange(config2.p), "i -> (i j)", j=2*config2.p) # shape (2*P*P,)
b_vector2 = einops.repeat(t.arange(config2.p), "j -> (i j)", i=2*config2.p) # shape (2*P*P,)
equals_vector2 = einops.repeat(t.tensor(config2.p), " -> (i j)", i=2*config2.p, j=config2.p) # shape (2*P*P,)
plus_vector = einops.repeat(t.tensor(config2.p + 1), " -> i", i=config2.p) # shape (P,)
min_vector = einops.repeat(t.tensor(config2.p + 2), " -> i", i=config2.p) # shape (P,)
operator_vector = einops.repeat(t.cat((plus_vector, min_vector)), "j -> (i j)", i=config2.p)# shape (2*P*P,)

In [ ]:
dataset2 = t.stack((a_vector2, b_vector2, operator_vector, equals_vector2), dim=1).to(config2.device) # shape(2*P*P, 4)

dataset2[:5], dataset2.shape

In [ ]:
# Vectorized method of modular addition and modular subtraction formula
a = dataset2[:, 0]
b = dataset2[:, 1]
op = dataset2[:, 2]
labels2 = (a + (op == 80) * b - (op == 81) * b) % config2.p # This generates the appropriate labels based on the operator value

labels2[0:79], labels2.shape, labels2.unique().size() # Confirming the size of the out vocabulary

In [ ]:
# # Remove operator token to see if the model will grok again
# new_dataset2 = dataset2[:, [0, 1, 3]]

# new_dataset2[:5], new_dataset2.shape

New Train / test split

In [ ]:
# Use training_frac split criteria and randomization of the dataset and labels (avoid particular pattern in training or testing)
t.manual_seed(config2.seed)
index2 = t.randperm(2*config2.p*config2.p) # shape (2*P*P,)
random_dataset2 = dataset2[index2] # randomized dataset
random_labels2 = labels2[index2]
train_data2, train_labels2 = random_dataset2[:config2.training_index], random_labels2[:config2.training_index]
test_data2, test_labels2 = random_dataset2[config2.training_index:], random_labels2[config2.training_index:]

train_data2.shape, train_labels2.shape, test_data2.shape, test_labels2.shape, \
train_data2[:5], train_labels2[:5], test_data2[:5], test_labels2[:5]

In [ ]:
# Determine amount of addition vs. subtraction distribution in train and test data
print(f"Amount of plus operator in training data: {round(((train_data2[:, 2] == config2.p+1).sum() / train_data2.shape[0]).item() * 100, 2)}%")
print(f"Amount of plus operator in test data: {round(((test_data2[:, 2] == config2.p+1).sum() / test_data2.shape[0]).item() * 100, 2)}%")

## Model Training

### Define Model

The mainline model of the experiment is a one-layer ReLU-activated transformer with learned positional embeddings, token embeddings with $ d = 128 $, $ 4 $ attention heads of dimension $ \frac {d} {4} = 32 $, and $ n = 512 $ hidden units in the MLP. LayerNorm was not used and the embed/unembed matrices were not tied.

The following code was copied from Neel Nanda's instruction video (https://www.youtube.com/watch?v=ob4vuiqG2Go&list=PL7m7hLIqA0hqsReJ0NYhyN3xiFQW9ko1h&index=2)

In [ ]:
model = HookedTransformer(model_cfg) # change configuration to model_cfg_2eq for dual task problem

In [ ]:
model.cfg.device # Model on GPU

This is also directly adapted from Neel's instructions. We are going to eleminate all the biases from the model (model parameters are weights + biases). We simply do not need biases for this exercise and it will make the workflow easier.

In [ ]:
for name, p in model.named_parameters():
    if "b_" in name:
        p.requires_grad = False

### Define Optimizer

Use all of the hyperparameters for the optimizer as specified in the configuration file. Use the specific optimizer for either the single task or dual task problem.

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=config.lr, weight_decay=config.wd, betas=config.betas) # Single task optimizer

In [ ]:
# optimizer_2eq = optim.AdamW(model.parameters(), lr=config2.lr, weight_decay=config2.wd, betas=config2.betas) # Dual task optimizer

### Training

Full train function can be found in the custom function section.

In [ ]:
# Call train function and train mainline model for specified amount of epochs (replace values with comments for dual task problem)
train_losses, test_losses, train_accuracies, test_accuracies, model_checkpoints, checkpoint_epochs = train(
    model,
    train_data, # train_data2
    train_labels, # train_labels2
    test_data, # test_data2
    test_labels, # test_labels2
    loss_fn,
    config # config2
)

### Saving data of training run

In [ ]:
# Save data using PTH_LOCATION. This saves data into content directly. You may then download and add to your google drive.
# Rest of the notebook then works with mounting drive and pulling that data back in so you can perform analysis on multiple runs
t.save(
    {
        "model": model.state_dict(),
        "config": model.cfg,
        "checkpoints": model_checkpoints,
        "checkpoint_epochs": checkpoint_epochs,
        "test_losses": test_losses,
        "train_losses": train_losses,
        "test_accuracies": test_accuracies,
        "train_accuracies": train_accuracies,
        "index": index # index2 for dual task problem
    },
    PTH_LOCATION)

## Loading Data
This code pulls data from google drive into colab notebook instantly.

In [ ]:
# Mount drive to gain access to data -> Accept conditions of pop-up windows and complete mounting
drive.mount("content")

In [ ]:
# Create a dictionary of dictionaries that stores all data
run_dicts = {}

# Create drive base (parent folder in google drive where data is stored)
drive_base = "insert_your_drivebase_here"

# Load data into dictionary
run_dicts["name"] = t.load(drive_base + "filename.pth", map_location=config2.device, weights_only=False)
# Build additional entries as you run more experiments

In [ ]:
# Inspect contents to see how many files were pulled in
for k, v in run_dicts.items():
    print(k)

Loading two equation datasets

In [ ]:
# Create a dictionary of dictionaries
run_dicts_two_eq = {}

# Load data into dictionary
run_dicts_two_eq["name"] = t.load(drive_base + "filename.pth", map_location=config2.device, weights_only=False)
# Build additional entries as you run more experiments

In [ ]:
# Inspect contents to see how many files were pulled in
for k, v in run_dicts_two_eq.items():
    print(k)

#### Extracting all the necessary data components

In [ ]:
# Extract all model data components for each run (comment sections out if you did not load certain experiments)
# The entry below is just an example of how to potentially structure your data

# Run 1
run_1_dict = run_dicts["name"]
model1sd = run_1_dict["model"]
checkpoints1 = run_1_dict["checkpoints"]
checkpoint_epochs1 = run_1_dict["checkpoint_epochs"]
test_losses1 = run_1_dict["test_losses"]
train_losses1 = run_1_dict["train_losses"]
test_accuracies1 = run_1_dict["test_accuracies"]
train_accuracies1 = run_1_dict["train_accuracies"]
index1 = run_1_dict["index"]
model_cfg1 = run_1_dict["config"]

## Plotting!
Using plotly and custom functions to replicate plots from paper.

First create datastructures for plotting

In [ ]:
# Create list of training and testing losses
losses_wd1 = [
    train_losses1[::100],
    train_losses2[::100],
    train_losses3[::100],
    test_losses1[::100],
    test_losses2[::100],
    test_losses3[::100]
]

In [ ]:
# Create list of training and testing accuracies
accuracies_wd1 = [
    train_accuracies1[::100],
    train_accuracies2[::100],
    train_accuracies3[::100],
    test_accuracies1[::100],
    test_accuracies2[::100],
    test_accuracies3[::100]
]

Running plot functions

In [ ]:
loss_wd1_fig = line(
    losses_wd1,
    x_range=config.epochs,
    xaxis="Epoch",
    yaxis="Log Loss",
    log_y=True,
    # title="Average train and test loss for weight decay 1 and one equation",
    line_labels=["Avg Train Loss", "Avg Test Loss"],
    height=350,
    width=600,
    legend_dict=dict(x=0.67, y=0.9),
    plot_average=True,
    show_seeds=False
)

loss_wd1_fig = add_vlines(loss_wd1_fig, [1400, 11000, 12700])
loss_wd1_fig.add_scatter(
    x=[700, 6200, 11850],
    y=[1, 1, 1],
    text=["I", "II", "III"],
    mode="text",
    textfont=dict(size=20),
    showlegend=False
)

loss_wd1_fig.show()

In [ ]:
# Save high res image
save_fig(loss_wd1_fig, "graph_name")

In [ ]:
acc_wd1_fig = line(
    accuracies_wd1,
    x_range=config.epochs,
    xaxis="Epoch",
    yaxis="Accuracy",
    # title="Average train and test accuracy for weight decay 1 and one equation",
    line_labels=["Avg Train Accuracy", "Avg Test Accuracy"],
    height=350,
    width=600,
    legend_dict=dict(x=0.6, y=0.9),
    plot_average=True
)

acc_wd1_fig.show()

In [ ]:
# Save high res image
save_fig(acc_wd1_fig, "graph_name")

In [ ]:
loss_wds_fig = line(
    [train_losses5[::100], train_losses2[::100], train_losses8[::100], test_losses5[::100], test_losses2[::100], test_losses8[::100]],
    xaxis="Epoch",
    yaxis="Log Loss",
    x_range=config.epochs,
    log_y=True,
    # title="Train and test loss for three weight decays for one equation problem",
    height=350,
    width=600,
    legend_dict=dict(x=0.75, y=0.8),
    plot_average=False,
    line_colors=["blue", "blue", "blue", "#d97706", "#d97706", "#d97706"]
)

loss_wds_fig.data[0].update(showlegend=False)
loss_wds_fig.data[1].update(line={"dash": "dash"}, showlegend=False)
loss_wds_fig.data[2].update(line={"dash": "dot"}, showlegend=False)
loss_wds_fig.data[3].update(name="Wd 0.5")
loss_wds_fig.data[4].update(line={"dash": "dash"}, name="Wd 1.0")
loss_wds_fig.data[5].update(line={"dash": "dot"}, name="Wd 3.0")

loss_wds_fig.show()

In [ ]:
# Save high res image
save_fig(loss_wds_fig, "graph_name")

In [ ]:
acc_wds_fig = line(
    [
        train_accuracies5[::100], 
        train_accuracies2[::100], 
        train_accuracies8[::100], 
        test_accuracies5[::100], 
        test_accuracies2[::100], 
        test_accuracies8[::100]
    ],
    xaxis="Epoch",
    yaxis="Accuracy",
    x_range=config.epochs,
    # title="Train and test accuracy for three weight decays for one equation problem",
    height=350,
    width=600,
    legend_dict=dict(x=0.75, y=0.8),
    plot_average=False,
    line_colors=["blue", "blue", "blue", "#d97706", "#d97706", "#d97706"]
)

acc_wds_fig.data[0].update(showlegend=False)
acc_wds_fig.data[1].update(line={"dash": "dash"}, showlegend=False)
acc_wds_fig.data[2].update(line={"dash": "dot"}, showlegend=False)
acc_wds_fig.data[3].update(name="Wd 0.5")
acc_wds_fig.data[4].update(line={"dash": "dash"}, name="Wd 1.0")
acc_wds_fig.data[5].update(line={"dash": "dot"}, name="Wd 3.0")

acc_wds_fig.show()

In [ ]:
# Save high res image
save_fig(acc_wds_fig, "graph_name")

### Plotting Two Equation Experiments

In [ ]:
# Create list of training and testing losses
losses_wd1_two_eq = [
    train_losses12[::100],
    train_losses14[::100],
    train_losses16[::100],
    test_losses12[::100],
    test_losses14[::100],
    test_losses16[::100]
]

In [ ]:
# Create list of training and testing accuracies
accuracies_wd1_two_eq = [
    train_accuracies12[::100],
    train_accuracies14[::100],
    train_accuracies16[::100],
    test_accuracies12[::100],
    test_accuracies14[::100],
    test_accuracies16[::100]
]

In [ ]:
loss_2eq = line(
    losses_wd1_two_eq,
    xaxis="Epoch",
    yaxis="Log Loss",
    x_range=config.epochs,
    log_y=True,
    # title="Train and test loss for weight decay 1 two equation problem",
    line_labels=["Avg Train Loss", "Avg Test Loss"],
    height=350,
    width=600,
    legend_dict=dict(x=0.2, y=0.3),
    plot_average=True
)

loss_2eq.show()

In [ ]:
# Save high res image
save_fig(loss_2eq, "graph_name")

In [ ]:
acc_2eq = line(
    accuracies_wd1_two_eq,
    xaxis="Epoch",
    yaxis="Accuracy",
    x_range=config.epochs,
    # title="Train and test loss for weight decay 1 two equation problem",
    line_labels=["Avg Train Accuracy", "Avg Test Accuracy"],
    height=350,
    width=600,
    legend_dict=dict(x=0.6, y=0.1),
    plot_average=True
)

acc_2eq.show()

In [ ]:
# Save high res image
save_fig(acc_2eq, "graph_name")

In [ ]:
# Plot comparing loss of one equation vs. two equation problem
loss_eq_comp = line(
    [test_losses1[::100], test_losses12[::100]],
    xaxis="Epoch",
    yaxis="Log Loss",
    x_range=config.epochs,
    log_y=True,
    # title="Train and test loss for weight decay 1 two equation problem",
    line_labels=["Single Task", "Dual Task"],
    height=350,
    width=600,
    legend_dict=dict(x=0.65, y=0.9),
    plot_average=False,
    line_colors=["blue", "#d97706"]
)

loss_eq_comp.show()

In [ ]:
# Save high res image
save_fig(loss_eq_comp, "graph_name")

In [ ]:
# Run 12 vs. run 13 comparison plot
loss_2eq_no_tok = line(
    [train_losses12[::100], train_losses13[::100], test_losses12[::100], test_losses13[::100]],
    xaxis="Epoch",
    yaxis="Log Loss",
    x_range=config.epochs,
    log_y=True,
    # title="Train and test loss for weight decay 1 two equation problem no op token comparison",
    height=350,
    width=600,
    legend_dict=dict(x=0.2, y=0.3),
    plot_average=False,
    line_colors=["blue", "blue", "#d97706", "#d97706"]
)

loss_2eq_no_tok.data[0].update(name="Train loss")
loss_2eq_no_tok.data[1].update(line={"dash": "dash"}, showlegend=False)
loss_2eq_no_tok.data[2].update(name="Test Loss")
loss_2eq_no_tok.data[3].update(line={"dash": "dash"}, showlegend=False)

loss_2eq_no_tok.show()

In [ ]:
# Save high res image
save_fig(loss_2eq_no_tok, "graph_name")

In [ ]:
# Run 12 vs. run 13 comparison plot
acc_2eq_no_tok = line(
    [train_accuracies12[::100], train_accuracies13[::100], test_accuracies12[::100], test_accuracies13[::100]],
    xaxis="Epoch",
    yaxis="Accuracy",
    x_range=config.epochs,
    # title="Train and test loss for weight decay 1 two equation problem no op token comparison",
    height=350,
    width=600,
    legend_dict=dict(x=0.6, y=0.55),
    plot_average=False,
    line_colors=["blue", "blue", "#d97706", "#d97706"]
)

acc_2eq_no_tok.data[0].update(name="Train Accuracy")
acc_2eq_no_tok.data[1].update(line={"dash": "dash"}, showlegend=False)
acc_2eq_no_tok.data[2].update(name="Test Accuracy")
acc_2eq_no_tok.data[3].update(line={"dash": "dash"}, showlegend=False)

acc_2eq_no_tok.show()

In [ ]:
# Save high res image
save_fig(acc_2eq_no_tok, "graph_name")

## Fourier Transforms
These following graphs are all model agnostic. The graphs are simulations of sine and cosine waves which are only dependent on the prime moduli.

In [ ]:
# Create the fourier basis, which consists of sin and cosine waves of 2*pi / P using a frequency range from 1 to P//2 + 1
# For P = 113, freq-range from 1 to 56
fourier_fig = imshow(fourier_basis, xaxis="Input", yaxis="Component", y=fourier_basis_names, height=400, width=800)
fourier_fig.show()

In [ ]:
# Plotting first 8 components of the fourierbase as function of the embedding input dimension (112, P - 1)
fourier_fig1_2 = line(
    fourier_basis[:8].T,
    xaxis="Input",
    line_labels=fourier_basis_names[:8],
    title="First 8 Fourier Components",
    height=400,
    width=800
)
fourier_fig1_2.show()

In [ ]:
# Plotting middle fourier components now, model learns these as the lookup table for the embeddings!
fourier_fig1_3 = line(
    fourier_basis[25:29].T,
    xaxis="Input",
    line_labels=fourier_basis_names[25:29],
    title="Middle Fourier Components",
    height=400,
    width=800
)
fourier_fig1_3.show()

The below graphs are the fourier basis for the dual task experiment with a different prime moduli

In [ ]:
# Create the fourier basis, which consists of sin and cosine waves of 2*pi / P using a frequency range from 1 to P//2 + 1
# For P = 79, freq-range from 1 to 39
fourier_fig2 = imshow(fourier_basis2, xaxis="Input", yaxis="Component", y=fourier_basis_names2, height=400, width=800)
fourier_fig2.show()

In [ ]:
# Plotting first 8 components of the fourierbase as function of the embedding input dimension (78, P - 1)
fourier_fig2_2 = line(
    fourier_basis2[:8].T,
    xaxis="Input",
    line_labels=fourier_basis_names2[:8],
    title="First 8 Fourier Components",
    height=400,
    width=800
)
fourier_fig2_2.show()

In [ ]:
# Plotting middle fourier components now, model learns these as the lookup table for the embeddings!
fourier_fig2_3 = line(
    fourier_basis2[25:29].T,
    xaxis="Input",
    line_labels=fourier_basis_names2[25:29],
    title="Middle Fourier Components",
    height=400,
    width=800
)
fourier_fig2_3.show()

## Analyzing the Model
This section is about exploring the model and understanding the observed Grokking behavior. We are going to use mechanistic interpretability principles to explore the inner workings of the model.
<br>
The following code has been adapted from (https://www.youtube.com/watch?v=o0FppeD_xXQ&list=PL7m7hLIqA0hqsReJ0NYhyN3xiFQW9ko1h&index=3)
<br>
Feedback from watching above video:
* Components of network that matter mostly are:
    * Attention pattern matmuls (1)
    * ReLU activation function of the MLP
    * Unembedding multiplied with MLP output matrix (2)
<br>
(1) $ W_{E} @ W_{V} @ W_{O} @ W_{in} $
<br>
(2) $ W_{out} @ W_{U} $

#### Re-create the model using state dictionary from training
Explore original loss on full dataset

In [ ]:
# Instantiate a new model
model1 = HookedTransformer(model_cfg1)

In [ ]:
# Load and process state dictionary into new model
model1.load_and_process_state_dict(model1sd)

In [ ]:
# Produce original logits and cache object with activations using full dataset
original_logits1, cache1 = model1.run_with_cache(dataset) # Dataset is (P*P, 3)
print(original_logits1.numel()) # Shape of original logits is (P*P, 3, P), numel() does 113*113*3*113 = 4,328,691

In [ ]:
# Original loss of the model generated from entire dataset, checking that model can do the problem now on full dataset
original_loss1 = loss_fn(original_logits1, labels).item()
print("Original Loss:", original_loss1) # Any loss below 1.0e-02 translates to perfect accuracy

#### Get Key Weight Matrices

In [ ]:
# Getting the most important weight matrices (see section above)
W_E1 = model1.embed.W_E[:-1] # Only need "=" - sign token to predict the next token (solution to modular addition problem)
W_neur1 = W_E1 @ model1.blocks[0].attn.W_V @ model1.blocks[0].attn.W_O @ model1.blocks[0].mlp.W_in
W_logit1 = model1.blocks[0].mlp.W_out @ model1.unembed.W_U

print("W_E1", W_E1.shape)
print("W_neur1", W_neur1.shape)
print("W_logit1", W_logit1.shape)

#### Get All Relevant Matmuls

In [ ]:
# Helper variables - use these in the next round of iteration
pattern_a1 = cache1["pattern", 0, "attn"][:, :, -1, 0]
neuron_acts1 = cache1["post", 0, "mlp"][:, -1, :] # post ReLU

In [ ]:
# Relevant matmuls for analysis
fourier_embed1 = fourier_basis @ W_E1
norm_fourier_embed1 = fourier_embed1.norm(dim=-1)
fourier_neuron_acts1 = fourier_basis @ einops.rearrange(neuron_acts1, "(a b) neuron -> neuron a b", a=config.p, b=config.p) @ fourier_basis.T
neuron_logit1 = W_logit1 @ fourier_basis.T
norm_neuron_logit1 = neuron_logit1.norm(dim=0)
W_logit_fourier1 = W_logit1 @ fourier_basis

In [ ]:
# Splitting key frequencies for embedding index by fourier_basis_names
norm_const_embed1 = norm_fourier_embed1[np.array(fourier_basis_names) == "Constant"]
norm_cos_embed1 = norm_fourier_embed1[np.char.startswith(np.array(fourier_basis_names), "cos")]
norm_sin_embed1 = norm_fourier_embed1[np.char.startswith(np.array(fourier_basis_names), "sin")]

norm_const_embed1_reshape = t.cat([norm_const_embed1, t.zeros_like(norm_cos_embed1)])
norm_cos_embed1_reshape = t.cat([t.zeros_like(norm_const_embed1), norm_cos_embed1])
norm_sin_embed1_reshape = t.cat([t.zeros_like(norm_const_embed1), norm_sin_embed1])

In [ ]:
# Splitting key frequencies for neuron-logit index by fourier_basis_names
norm_const_logit1 = norm_neuron_logit1[np.array(fourier_basis_names) == "Constant"]
norm_cos_logit1 = norm_neuron_logit1[np.char.startswith(np.array(fourier_basis_names), "cos")]
norm_sin_logit1 = norm_neuron_logit1[np.char.startswith(np.array(fourier_basis_names), "sin")]

norm_const_logit1_reshape = t.cat([norm_const_logit1, t.zeros_like(norm_cos_logit1)])
norm_cos_logit1_reshape = t.cat([t.zeros_like(norm_const_logit1), norm_cos_logit1])
norm_sin_logit1_reshape = t.cat([t.zeros_like(norm_const_logit1), norm_sin_logit1])

#### Getting Key Frequencies
This section jumps ahead a bit, but it allows for skipping some of the mech interp part and running the third section for quicker analysis

In [ ]:
# Using fourier_basis matrix multiplication with the embedding to figure out key frequencies (normalizing over d_model)
embed_fourier_run1 = bar(
    [norm_const_embed1_reshape, norm_cos_embed1_reshape, norm_sin_embed1_reshape],
    xaxis="Frequency k",
    yaxis="Norm of Fourier Component",
    bar_labels=["Const", "Cos", "Sin"],
    height=350,
    width=600,
    bar_colors=["green", "#d97706", "blue"],
    legend_dict=dict(x=0.75, y=0.9)
)
embed_fourier_run1.update_layout(
    title={
        "text": "<b>Fourier Components of Embedding Matrix</b>",
        "x": 0.13,
        "y": 0.9,
    }
)

In [ ]:
# Save high res image
save_fig(embed_fourier_run1, "graph_name")

In [ ]:
# Threshold of 3 chosen, this value may need to be adjusted depending on specific run
key_freq_indices1 = t.nonzero(norm_fourier_embed1 > 3, as_tuple=True)[0].tolist()
key_freq_indices1

In [ ]:
# Splitting key frequency indices by cos and sin functions
key_freq_indices_cos1 = [k for k in key_freq_indices1 if "cos" in fourier_basis_names[k]]
key_freq_indices_sin1 = [k for k in key_freq_indices1 if "sin" in fourier_basis_names[k]]

print(key_freq_indices_cos1, key_freq_indices_sin1)

In [ ]:
# Key frequencies extracted from the indices based on fourier names (this will be adjusted automatically from models)
key_freqs1 = sorted({int(s.split()[1]) for s in [fourier_basis_names[i] for i in key_freq_indices1]})
key_freqs1

#### Looking at Attention Heads

In [ ]:
# Attention pattern for head with specific index from a-token paid to "=" - sign
attn_head0_ato_run1 = imshow(
    einops.rearrange(pattern_a1[:, 0], "(a b) -> a b", a=config.p, b=config.p),
    # title="Attention first Head from a -> =",
    xaxis="a",
    yaxis="b",
    height=460,
    width=500
)

attn_head0_ato_run1.update_layout(
    title={
        "text": "<b>Attention Score for Head 0</b>",
        "x": 0.16,
        "y": 0.93,
        "font": dict(size=20)
    }
)

In [ ]:
# Save high res image
save_fig(attn_head0_ato_run1, "graph_name")

#### Singular Value Decomposition (SVD) of the embedding matrix

In [ ]:
U1, S1, Vh1 = t.svd(W_E1) # Specific operation to convert matrix into orthogonal and diagonal matrices to help reduce dimensions, S is ordered

In [ ]:
# Observing periodicity in the embedding structure
princomp_embed_run1 = line(
    U1[:, 0],
    xaxis="Vocabulary",
    yaxis="",
    height=350,
    width=600
)
princomp_embed_run1.update_layout(
    title={
        "text": "<b>First Principal Component of Embedding Matrix</b>",
        "x": 0.13,
        "y": 0.9
    },
    showlegend=False
)

In [ ]:
# Save high res image
save_fig(princomp_embed_run1, "graph_name")

### Analyze Embedding

In [ ]:
# Embedding analysis, key frequencies the model has learned appear in this graph for the first time (key freqs extracted above)
fourier_embed_run1 = imshow(
    fourier_embed1,
    yaxis="Fourier Component",
    xaxis="Residual Stream",
    y=fourier_basis_names,
    height=460,
    width=500
)

fourier_embed_run1.update_layout(
    title={
        "text": "<b>Embedding in Fourier Basis</b>",
        "x": 0.18,
        "y": 0.93,
        "font": dict(size=20)
    }
)

In [ ]:
# Save high res image
save_fig(fourier_embed_run1, "graph_name")

### Analyze Neurons

In [ ]:
# Neuron activations post-ReLU (also noticing a distinct pattern, seems periodic)
neur_acts1_run1 = imshow(
    einops.rearrange(neuron_acts1[:, 1], "(a b) -> a b", a=config.p, b=config.p), # Second neuron
    xaxis="a",
    yaxis="b",
    height=460,
    width=500
)
neur_acts1_run1.update_layout(
    title={
        "text": "<b>Activations for Neuron 1</b>",
        "x": 0.16,
        "y": 0.93,
        "font": dict(size=20)
    }
)

In [ ]:
# Save high res image
save_fig(neur_acts1_run1, "graph_name")

In [ ]:
# Neuron activations post-ReLU (also noticing a distinct pattern, seems periodic)

second_neuron = einops.rearrange(neuron_acts1[:, 1], "(a b) -> a b", a=config.p, b=config.p)
third_neuron = einops.rearrange(neuron_acts1[:, 2], "(a b) -> a b", a=config.p, b=config.p)

neuron_inter = second_neuron - third_neuron

neur_inter_run1 = imshow(
    neuron_inter,
    xaxis="a",
    yaxis="b",
    height=460,
    width=500
)

neur_inter_run1.update_layout(
    title={
        "text": "<b>Neuron 1 and 2 Interference</b>",
        "x": 0.16,
        "y": 0.93,
        "font": dict(size=20)
    }
)

In [ ]:
# Save high res image
save_fig(neur_inter_run1, "graph_name")

In [ ]:
# The following graph attempts to show the construction of the learned trig identities in the MLP
# First we will show the cos(key_freq*a) * cos(key_freq*b)
kf = key_freqs1[-1]

neur_fourier_run1 = imshow(
    fourier_basis[key_freq_indices_cos1[-1]][None, :] * fourier_basis[key_freq_indices_cos1[-1]][:, None],
    # title=f"cos {kf}a * cos {kf}b",
    xaxis="a",
    yaxis="b",
    height=460,
    width=510
)

neur_fourier_run1.update_layout(
    title={
        "text": f"<b>Cos {kf}a * Cos {kf}b</b>",
        "x": 0.16,
        "y": 0.93,
        "font": dict(size=20)
    }
)

In [ ]:
# Save high res image
save_fig(neur_fourier_run1, "graph_name")

In [ ]:
# Next we will look at the sin(key_freq*a) * cos(key_freq*b) + cos(key_freq*a) * sin(key_freq*b) identity
first_component = fourier_basis[key_freq_indices_cos1[-1]][None, :] * fourier_basis[key_freq_indices_cos1[-1]][:, None]
second_component = fourier_basis[key_freq_indices_sin1[-1]][None, :] * fourier_basis[key_freq_indices_sin1[-1]][:, None]
sin_trig_identity = first_component - second_component

neur_fourier_inter_run1 = imshow(
    sin_trig_identity,
    # title=f"cos {kf}a * cos {kf}b - sin {kf}a * sin {kf}b",
    xaxis="a",
    yaxis="b",
    height=460,
    width=510
)

neur_fourier_inter_run1.update_layout(
    title={
        "text": f"<b>Cos {kf}a * Cos {kf}b - Sin {kf}a * Sin {kf}b </b>",
        "x": 0.16,
        "y": 0.93
    }
)

In [ ]:
# Save high res image
save_fig(neur_fourier_inter_run1, "graph_name")

### Neuron Clusters

In [ ]:
# Centering fourier_neuron_acts by removing the mean
fourier_neuron_acts1[:, 0, 0] = 0.
print("fourier_neuron_acts1", fourier_neuron_acts1.shape)

In [ ]:
neuron_freq_norm1 = t.zeros(config.p//2, model1.cfg.d_mlp).to(config.device)
for freq in range(0, config.p//2):
    for x in [0, 2*(freq+1) - 1, 2*(freq+1)]:
        for y in [0, 2*(freq+1) - 1, 2*(freq+1)]:
            neuron_freq_norm1[freq] += fourier_neuron_acts1[:, x, y]**2
neuron_freq_norm1 = neuron_freq_norm1 / fourier_neuron_acts1.pow(2).sum(dim=[-1, -2])[None, :]

# This graph shows which neurons are explained by the key frequencies
neur_frac_run1 = imshow(
    neuron_freq_norm1,
    xaxis="Neuron",
    yaxis="Frequency k",
    y=t.arange(1, config.p//2+1),
    height=460,
    width=500
)

neur_frac_run1.update_layout(
    title={
        "text": "<b>Neuron Explained by Frequency</b>",
        "x": 0.16,
        "y": 0.93
    }
)

In [ ]:
# Save high res image
save_fig(neur_frac_run1, "graph_name")

### Read Off Neuron-Logit Weights to Interpret

In [ ]:
neur_logit_run1 = bar(
    [norm_const_logit1_reshape, norm_cos_logit1_reshape, norm_sin_logit1_reshape],
    xaxis="Frequency k",
    yaxis="Norm of Fourier Component",
    bar_labels=["Const", "Cos", "Sin"],
    height=350,
    width=600,
    bar_colors=["green", "#d97706", "blue"],
    legend_dict=dict(x=0.75, y=0.9)
)
neur_logit_run1.update_layout(
    title={
        "text": "<b>Fourier Components of Neuron-Logit Map</b>",
        "x": 0.13,
        "y": 0.9,
    }
)

In [ ]:
# Save high res image
save_fig(neur_logit_run1, "graph_name")

## Progress Measures

### Defining Progress Measures

Setting up some variables for the next few experiments

In [ ]:
metric_cache1 = {}

memorization_end_epoch1 = 1400
circuit_formation_end_epoch1 = 11000
cleanup_end_epoch1 = 12700

xpos1 = [memorization_end_epoch1, circuit_formation_end_epoch1, cleanup_end_epoch1]

#### Logit Periodicity

In [ ]:
get_metrics(model1, checkpoints1, metric_cache1, get_cos_coeffs, cos_cube, dataset, config, name="cos_coeffs")
print(metric_cache1["cos_coeffs"].shape)

In [ ]:
# This figure shows the coefficients of the cosine similarity
cos_coeffs_run1 = line(
    metric_cache1["cos_coeffs"][:, [17, 19, 31]],
    line_labels=key_freqs1,
    xaxis="Epoch",
    x_range=config.epochs,
    yaxis="",
    height=350,
    width=600,
    start_epoch=checkpoint_epochs1[0],
    legend_dict=dict(x=0.2, y=0.95),
)

cos_coeffs_run1 = add_vlines(cos_coeffs_run1, xpos1)
cos_coeffs_run1.update_layout(
    title={
        "text": "<b>Coefficients of cos(wk(a+b-c)) in the Logits </b>",
        "x": 0.13,
        "y": 0.9
    },
    legend_title_text="Freq"
)

In [ ]:
# Save high res image
save_fig(cos_coeffs_run1, "graph_name")

#### Restricted Loss

In [ ]:
get_metrics(
    model1,
    checkpoints1,
    metric_cache1,
    get_restricted_loss,
    dataset,
    key_freqs1,
    index1,
    labels,
    config,
    name="restricted_loss",
    reset=True
)
print(metric_cache1["restricted_loss"].shape)

#### Excluded Loss

In [ ]:
get_metrics(
    model1,
    checkpoints1,
    metric_cache1,
    get_excluded_loss,
    dataset,
    key_freqs1,
    index1,
    labels,
    config,
    name="excluded_loss",
    reset=True
)
print(metric_cache1["excluded_loss"].shape)

In [ ]:
rest_excl_loss_run1 = line(
    [train_losses1[99::100], test_losses1[99::100], metric_cache1["excluded_loss"], metric_cache1["restricted_loss"]],
    x_range=config.epochs,
    xaxis="Epoch",
    yaxis="Log Loss",
    log_y=True,
    line_labels=["Train Loss", "Test Loss", "Excluded Loss", "Restricted Loss"],
    height=350,
    width=600,
    legend_dict=dict(x=0.67, y=0.55),
    line_colors=["blue", "#d97706", "green", "red"],
    start_epoch=checkpoint_epochs1[0]
)

rest_excl_loss_run1 = add_vlines(rest_excl_loss_run1, xpos1)
rest_excl_loss_run1.update_layout(
    title={
        "text": "<b>Excluded and Restricted Loss</b>",
        "x": 0.13,
        "y": 0.9
    }
)

In [ ]:
# Save high res image
save_fig(rest_excl_loss_run1, "graph_name")

## Two Equation Problem Mech Interp

#### Re-create the model using state dictionary from training
Explore original loss on full dataset

In [ ]:
# Instantiate a new model
model12 = HookedTransformer(model_cfg12)

In [ ]:
# Load and process state dictionary into new model
model12.load_and_process_state_dict(model12sd)

In [ ]:
# Produce original logits and cache object with activations using full dataset
original_logits12, cache12 = model12.run_with_cache(dataset2) # Dataset is (2*P*P, 4)
print(original_logits12.numel()) # Shape of original logits is (2*P*P, 4, P), numel() does 79*79*4*79 = 3,944,312

In [ ]:
# Original loss of the model generated from entire dataset, checking that model can do the problem now on full dataset
original_loss12 = loss_fn(original_logits12, labels2).item()
print("Original Loss:", original_loss12) # Any loss below 1.0e-02 translates to perfect accuracy

#### Get Key Weight Matrices

In [ ]:
# Getting the most important weight matrices (see section above)
W_E12 = model12.embed.W_E[:config2.p] # Only need "=" - sign token to predict the next token (solution to both problems)
W_neur12 = W_E12 @ model12.blocks[0].attn.W_V @ model12.blocks[0].attn.W_O @ model12.blocks[0].mlp.W_in
W_logit12 = model12.blocks[0].mlp.W_out @ model12.unembed.W_U

print("W_E12", W_E12.shape)
print("W_neur12", W_neur12.shape)
print("W_logit12", W_logit12.shape)

#### Get All Relevant Matmuls

In [ ]:
# # Helper variables - use these in the next round of iteration
pattern_a12 = cache12["pattern", 0, "attn"][:, :, -1, 0] # a -> =
neuron_acts12 = cache12["post", 0, "mlp"][:, -1, :] # post ReLU, focusing on =-token

In [ ]:
# Relevant matmuls for analysis
fourier_embed12 = fourier_basis2 @ W_E12
norm_fourier_embed12 = fourier_embed12.norm(dim=-1)
neuron_logit12 = W_logit12 @ fourier_basis2.T
norm_neuron_logit12 = neuron_logit12.norm(dim=0)
W_logit_fourier12 = W_logit12 @ fourier_basis2

In [ ]:
# Splitting key frequencies for embedding index by fourier_basis_names2
norm_const_embed12 = norm_fourier_embed12[np.array(fourier_basis_names2) == "Constant"]
norm_cos_embed12 = norm_fourier_embed12[np.char.startswith(np.array(fourier_basis_names2), "cos")]
norm_sin_embed12 = norm_fourier_embed12[np.char.startswith(np.array(fourier_basis_names2), "sin")]

norm_const_embed12_reshape = t.cat([norm_const_embed12, t.zeros_like(norm_cos_embed12)])
norm_cos_embed12_reshape = t.cat([t.zeros_like(norm_const_embed12), norm_cos_embed12])
norm_sin_embed12_reshape = t.cat([t.zeros_like(norm_const_embed12), norm_sin_embed12])

In [ ]:
# Splitting key frequencies for neuron-logit index by fourier_basis_names2
norm_const_logit12 = norm_neuron_logit12[np.array(fourier_basis_names2) == "Constant"]
norm_cos_logit12 = norm_neuron_logit12[np.char.startswith(np.array(fourier_basis_names2), "cos")]
norm_sin_logit12 = norm_neuron_logit12[np.char.startswith(np.array(fourier_basis_names2), "sin")]

norm_const_logit12_reshape = t.cat([norm_const_logit12, t.zeros_like(norm_cos_logit12)])
norm_cos_logit12_reshape = t.cat([t.zeros_like(norm_const_logit12), norm_cos_logit12])
norm_sin_logit12_reshape = t.cat([t.zeros_like(norm_const_logit12), norm_sin_logit12])

#### Getting Key Frequencies

In [ ]:
# Using fourier_basis matrix multiplication with the embedding to figure out key frequencies (normalizing over d_model)
embed_fourier_run12 = bar(
    [norm_const_embed12_reshape, norm_cos_embed12_reshape, norm_sin_embed12_reshape],
    xaxis="Frequency k",
    yaxis="Norm of Fourier Component",
    bar_labels=["Const", "Cos", "Sin"],
    height=350,
    width=600,
    bar_colors=["green", "#d97706", "blue"],
    legend_dict=dict(x=0.8, y=0.95)
)
embed_fourier_run12.update_layout(
    title={
        "text": "<b>Fourier Components of Embedding Matrix</b>",
        "x": 0.13,
        "y": 0.9,
    }
)

In [ ]:
# Save high res image
save_fig(embed_fourier_run12, "graph_name")

In [ ]:
# Threshold of 3 chosen, this value may need to be adjusted depending on specific run
key_freq_indices12 = t.nonzero(norm_fourier_embed12 > 3, as_tuple=True)[0].tolist()
key_freq_indices12

In [ ]:
# Key frequencies extracted from the indices based on fourier names (this will be adjusted automatically from models)
key_freqs12 = sorted({int(s.split()[1]) for s in [fourier_basis_names2[i] for i in key_freq_indices12]})
key_freqs12

#### Looking at Attention Heads

In [ ]:
# Attention pattern for head with specific index from a-token paid to "=" - sign
attn_head0_ato_batch1_run12 = imshow(
    einops.rearrange(pattern_a12[:config2.p*config2.p, 0], "(a b) -> a b", a=config2.p, b=config2.p),
    # title="Attention first Head from a -> =",
    xaxis="a",
    yaxis="b",
    height=460,
    width=510
)

attn_head0_ato_batch1_run12.update_layout(
    title={
        "text": "<b>Attention Score for Head 0</b>",
        "x": 0.16,
        "y": 0.93,
        "font": dict(size=20)
    }
)

In [ ]:
# Save high res image
save_fig(attn_head0_ato_batch1_run12, "graph_name")

#### Singular Value Decompostion (SVD) of the Embedding Matrix

In [ ]:
U12, S12, Vh12 = t.svd(W_E12) # Specific operation to convert matrix into orthogonal and diagonal matrices to help reduce dimensions, S is ordered

In [ ]:
# Observing periodicity in the embedding structure
princomp_embed_run12 = line(
    U12[:, 0],
    xaxis="Vocabulary",
    yaxis="",
    height=350,
    width=600
)
princomp_embed_run12.update_layout(
    title={
        "text": "<b>First Principal Component of Embedding Matrix</b>",
        "x": 0.13,
        "y": 0.9
    },
    showlegend=False
)

In [ ]:
# Save high res image
save_fig(princomp_embed_run12, "graph_name")

#### Analyze the Embedding

In [ ]:
# Embedding analysis, key frequencies the model has learned appear in this graph for the first time (key freqs extracted above)
fourier_embed_run12 = imshow(
    fourier_embed12,
    yaxis="Fourier Component",
    xaxis="Residual Stream",
    y=fourier_basis_names2,
    height=460,
    width=500
)

fourier_embed_run12.update_layout(
    title={
        "text": "<b>Embedding in Fourier Basis</b>",
        "x": 0.18,
        "y": 0.93,
        "font": dict(size=20)
    }
)

In [ ]:
# Save high res image
save_fig(fourier_embed_run12, "graph_name")

#### Analyze Neurons

In [ ]:
# Neuron activations post-ReLU (also noticing a distinct pattern, seems periodic)
neur_acts12_batch1_run12 = imshow(
    einops.rearrange(neuron_acts12[:config2.p*config2.p, 6], "(a b) -> a b", a=config2.p, b=config2.p), # seventh neuron
    xaxis="a",
    yaxis="b",
    height=460,
    width=520
)
neur_acts12_batch1_run12.update_layout(
    title={
        "text": "<b>Activations for Neuron 6</b>",
        "x": 0.16,
        "y": 0.93,
        "font": dict(size=20)
    }
)

In [ ]:
# Save high res image
save_fig(neur_acts12_batch1_run12, "graph_name")

#### Neuron-Logits Weights key frequencies

In [ ]:
neur_logit_run12 = bar(
    [norm_const_logit12_reshape, norm_cos_logit12_reshape, norm_sin_logit12_reshape],
    xaxis="Frequency k",
    yaxis="Norm of Fourier Component",
    bar_labels=["Const", "Cos", "Sin"],
    height=350,
    width=600,
    bar_colors=["green", "#d97706", "blue"],
    legend_dict=dict(x=0.8, y=0.95)
)
neur_logit_run12.update_layout(
    title={
        "text": "<b>Fourier Components of Neuron-Logit Map</b>",
        "x": 0.13,
        "y": 0.9,
    }
)

In [ ]:
# Save high res image
save_fig(neur_logit_run12, "graph_name")

#### Progress measures
Only exploring coefficients of cosine similarity (excluded and restricted loss currently do not work with dual task set-up)

In [ ]:
metric_cache12 = {}

memorization_end_epoch12 = 1500
circuit_formation_end_epoch12 = 8700
cleanup_end_epoch12 = 16500

xpos12 = [memorization_end_epoch12, circuit_formation_end_epoch12, cleanup_end_epoch12]

In [ ]:
# First batch cosine similarity using the modular addition task
get_metrics(model12, checkpoints12, metric_cache12, get_cos_coeffs, cos_cube2, dataset2[:config2.p*config2.p], config2, name="cos_coeffs")
print(metric_cache12["cos_coeffs"].shape)

In [ ]:
# Second batch cosine similarity using the modular subtraction task
get_metrics(model12, checkpoints12, metric_cache12, get_cos_coeffs, cos_cube3, dataset2[config2.p*config2.p:], config2, name="cos_coeffs2")
print(metric_cache12["cos_coeffs2"].shape)

In [ ]:
# This figure shows the coefficients of the cosine similarity

data_list = [metric_cache12["cos_coeffs"][:, [7, 27, 28]], metric_cache12["cos_coeffs2"][:, [7, 27, 28]]]
cos_coeffs_run12 = line(
    t.cat(data_list, dim=1),
    xaxis="Epoch",
    x_range=config2.epochs,
    yaxis="",
    height=350,
    width=600,
    start_epoch=checkpoint_epochs12[0],
    legend_dict=dict(x=0.15, y=0.2),
)

cos_coeffs_run12 = add_vlines(cos_coeffs_run12, xpos12)
cos_coeffs_run12.update_layout(
    title={
        "text": "<b>Coefficients of cos(wk(a+b-c)) and <br>cos(wk(a-b-c)) in the Logits </b>",
        "x": 0.13,
        "y": 0.92
    },
    legend_title_text="Freq"
)

cos_coeffs_run12.data[0].update(name="8")
cos_coeffs_run12.data[1].update(name="28")
cos_coeffs_run12.data[2].update(name="29")
cos_coeffs_run12.data[3].update(line={"dash": "dot", "color": f"{cos_coeffs_run12.data[0].line.color}"}, showlegend=False)
cos_coeffs_run12.data[4].update(line={"dash": "dot", "color": f"{cos_coeffs_run12.data[1].line.color}"}, showlegend=False)
cos_coeffs_run12.data[5].update(line={"dash": "dot", "color": f"{cos_coeffs_run12.data[2].line.color}"}, showlegend=False)

cos_coeffs_run12.show()

In [ ]:
# Save high res image
save_fig(cos_coeffs_run12, "graph_name")

In [ ]:
metric_cache12["cos_coeffs"][-1, [7, 27, 28]]

In [ ]:
metric_cache12["cos_coeffs2"][-1, [7, 27, 28]]